In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.layers import Conv2D, Dense, BatchNormalization, Activation, Dropout, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from keras.callbacks import ModelCheckpoint,EarlyStopping
import datetime
from keras import regularizers
import matplotlib.pyplot as plt
from keras.utils.vis_utils import plot_model

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time

In [4]:
DATADIR = "../input/facial-expression-recog-image-ver-of-fercdataset/Dataset/"
# ../input/facial-expression-recog-image-ver-of-fercdataset/Dataset/test
CLASSES = ["anger", "disgust", "fear", "happiness", "neutral", "sadness", "surprise"]

training_data = []
test_data = []

def create_training_data():
    for category in CLASSES:
        path = os.path.join(DATADIR, "train", category)
        class_num = CLASSES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (48, 48))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass

def create_test_data():
    for category in CLASSES:
        path = os.path.join(DATADIR, "test", category)
        class_num = CLASSES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (48, 48))
                test_data.append([new_array, class_num])
            except Exception as e:
                pass
            
create_training_data()
create_test_data()

In [5]:
print(len(training_data))
print(len(test_data))

0
0


In [6]:
import random

random.shuffle(training_data)
random.shuffle(test_data)

X_train = []
y_train = []

X_test = []
y_test = []

for features, label in training_data:
    X_train.append(features)
    y_train.append(label)

for features, label in test_data:
    X_test.append(features)
    y_test.append(label)

X_train = np.array(X_train).reshape(-1, 48, 48, 1)
X_test = np.array(X_test).reshape(-1, 48, 48, 1)

X_train = X_train/255.0
X_test = X_test/255.0

In [7]:
print(X_train.shape[1:])
print(X_test.shape[1:])

(48, 48, 1)
(48, 48, 1)


In [8]:
path = '../input/facial-expression-recog-image-ver-of-fercdataset/Dataset'
train_image_files_path = os.path.join(path, "train/")
valid_image_files_path = os.path.join(path, "test/")
train_image_files_path

'../input/facial-expression-recog-image-ver-of-fercdataset/Dataset/train/'

In [9]:
##
img_width = 48
img_height = 48
target_size = (img_width, img_height)
#training_images = np.expand_dims(training_images, axis=0)
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(train_image_files_path,  
                                                        color_mode = "grayscale",
                                                        target_size = target_size,
                                                        class_mode = 'categorical',
#                                                         classes = CLASSES,
                                                        seed = 42)
test_set = test_datagen.flow_from_directory(valid_image_files_path,
                                                    color_mode = "grayscale",
                                                    target_size = target_size,
                                                    class_mode = 'categorical',
#                                                     classes = CLASSES,
                                                    seed = 42)

# test_set = test_datagen.flow_from_directory(
#     'C:/Users/anude/Downloads/archive/Dataset/test',
#     target_size=(48, 48),
#     batch_size=32,
#     class_mode='categorical')


Found 32298 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


In [10]:
weight_decay = 1e-4

num_classes = 7

model = tf.keras.models.Sequential()

model.add(Conv2D(64, (4,4), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=(48,48,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (4,4), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(128, (4,4), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (4,4), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (4,4), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation="linear"))
model.add(Activation('elu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(0.0003), metrics=['accuracy'])
 
model.summary()

2022-11-14 07:11:20.964007: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 07:11:21.131208: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 07:11:21.132386: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-14 07:11:21.133974: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 64)        1088      
_________________________________________________________________
activation (Activation)      (None, 48, 48, 64)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 64)        65600     
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 64)        0

In [11]:
X_train.shape[0]

0

In [12]:
history = model.fit(x=training_set,
                 validation_data=test_set,
                 epochs=30)

2022-11-14 07:11:24.521974: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/30


2022-11-14 07:11:26.500074: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


1010/1010 [==============================] - 161s 152ms/step - loss: 1.7616 - accuracy: 0.3336 - val_loss: 1.5542 - val_accuracy: 0.4294
Epoch 2/30
1010/1010 [==============================] - 48s 47ms/step - loss: 1.5329 - accuracy: 0.4269 - val_loss: 1.3878 - val_accuracy: 0.4921
Epoch 3/30
1010/1010 [==============================] - 46s 46ms/step - loss: 1.4037 - accuracy: 0.4783 - val_loss: 1.3111 - val_accuracy: 0.5157
Epoch 4/30
1010/1010 [==============================] - 46s 46ms/step - loss: 1.3142 - accuracy: 0.5168 - val_loss: 1.2287 - val_accuracy: 0.5486
Epoch 5/30
1010/1010 [==============================] - 46s 46ms/step - loss: 1.2510 - accuracy: 0.5450 - val_loss: 1.2192 - val_accuracy: 0.5517
Epoch 6/30
1010/1010 [==============================] - 46s 46ms/step - loss: 1.2082 - accuracy: 0.5638 - val_loss: 1.1470 - val_accuracy: 0.5868
Epoch 7/30
1010/1010 [==============================] - 47s 47ms/step - loss: 1.1729 - accuracy: 0.5823 - val_loss: 1.1499 - val_accu